In [7]:
import pandas as pd

In [8]:
train_data_small = pd.read_csv('../data/raw/train_data_small.csv')
train_data = pd.read_csv('../data/raw/train_data.csv')

C:\Users\dh107\AppData\Local\Temp\ipykernel_18396\144004549.py:2: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('../data/raw/train_data.csv')


In [9]:
def print_data(df):
    print(f"data shape = {df.shape}")
    print(f"{df.columns}")

print_data(train_data_small)

data shape = (3402, 45)
Index(['Disease ID', 'Disease Name', 'Gene ID', 'UniProt_ID', 'GO_Terms',
       'PDB_IDs', 'PubMed_IDs', 'Protein_ID_Formatted', 'protein1', 'protein2',
       'combined_score', 'entryId', 'gene_x', 'geneSynonyms',
       'isReferenceProteome', 'isReviewed', 'sequenceChecksum',
       'sequenceVersionDate', 'uniprotAccession', 'uniprotId',
       'uniprotDescription', 'taxId', 'organismScientificName',
       'globalMetricValue', 'uniprotStart', 'uniprotEnd', 'uniprotSequence',
       'modelCreatedDate', 'organismCommonNames', 'proteinFullNames',
       'latestVersion', 'allVersions', 'isAMdata', 'organismScientificNameT',
       '_version_', 'proteinShortNames', 'uniprotAccession_unchar',
       'entry_name', 'protein_name', 'organism', 'tax_id', 'gene_y',
       'protein_existence', 'sequence_version', 'sequence'],
      dtype='object')


데이터를 보면 중복된 컬럼들이 보인다.(UniProt_ID, uniprotId, uniprotAccession 등)
따라서 아래와 같은 이유로 중복 및 학습에 불필요한 컬럼은 제거하여 학습에 사용한다.
제거된 데이터는 따로 `processed_train.csv`, `processed_train_small.csv` 로 저장한다.

| 컬럼명                       | 삭제 이유 (불필요성의 근거)                                              |
|-----------------------------|--------------------------------------------------------------|
| `PubMed_IDs`                | 논문 참조 ID로, 모델 예측에는 의미가 없으며 외부 메타정보에 해당                      |
| `entryId`                   | 데이터베이스의 내부 식별자로 분석이나 모델링에서 생물학적 의미 없음                     |
| `sequenceChecksum`          | 단백질 서열의 검증용 해시값으로, 서열 원본 정보 존재 시 불필요                        |
| `sequenceVersionDate`       | 서열 업데이트 날짜로 예측 모델에 활용 불가능 (생물학적 의미 X)                        |
| `uniprotDescription`        | 긴 텍스트 설명이며, 이미 단백질 이름 등 더 명확한 정보 존재 (중복 및 불필요)            |
| `organismScientificName`    | 모든 데이터가 인간 단백질이라면 동일값으로 의미 없음                                 |
| `globalMetricValue`         | 데이터베이스에서의 품질 점수 등으로, 생물학적 의미 없음                               |
| `uniprotStart`, `uniprotEnd`| 단백질 서열의 시작과 끝 위치는 서열 정보가 존재하면 불필요                           |
| `uniprotSequence`           | 서열 정보가 이미 `sequence` 컬럼에 존재하므로 중복                                  |
| `modelCreatedDate`,<br>`latestVersion`, `allVersions`, `_version_` | 데이터베이스 관리용 메타정보로 생물학적 모델 예측과 무관 |
| `proteinShortNames`,<br>`entry_name`, `protein_name`              | 이미 `proteinFullNames` 컬럼에 충분히 명시되어 중복 |
| `uniprotAccession_unchar`   | 원본 `uniprotAccession`의 특수문자 제거 형태로 중복 정보                            |
| `organism`                  | 생물학적 의미가 명확한 `organismScientificName`과 중복이며 데이터가 인간이라면 의미 없음 |
| `protein_existence`         | 단백질의 존재 여부는 일반적으로 모두 존재한다고 가정하며 의미가 낮음                  |
| `sequence_version`          | 서열의 버전 정보로 생물학적 예측과 무관                                            |
| `UniProt_ID`, `uniprotId`   | `uniprotAccession` 컬럼과 중복되며, 특히 isoform 등으로 인해 대표성 및 일관성 떨어짐  |
| `gene_x`                    | 결측치가 많아 유전자 정보로서 신뢰도 낮음                                          |
| `gene_y`                    | `Gene ID`와 완벽히 중복되므로 불필요한 컬럼                                       |
| 그 외 나머지                    | 다른 컬럼과 중복                                     |

In [10]:
# protein id 부분에 있는 "9606." 문자열 제거
def processing_protein_and_save(df):
    df["protein1"] = df["protein1"].str.replace("9606.", "", regex=False)
    df["protein2"] = df["protein2"].str.replace("9606.", "", regex=False)

# df, main_path: 드롭 후 최종 테이블 저장 경로)
def delete_columns_and_save(df, main_path):
    # 삭제할 컬럼들 정의
    cols_to_drop = [
        'PubMed_IDs', 'entryId', 'sequenceChecksum', 'sequenceVersionDate',
        'uniprotDescription', 'organismScientificName', 'globalMetricValue',
        'uniprotStart', 'uniprotEnd', 'uniprotSequence', 'modelCreatedDate',
        'latestVersion', 'allVersions', '_version_', 'proteinShortNames',
        'uniprotAccession_unchar', 'entry_name', 'protein_name', 'organism',
        'protein_existence', 'sequence_version', 'UniProt_ID', 'uniprotId',
        'gene_x', 'gene_y', 'organismScientificNameT', 'tax_id', 'taxId', 'organismCommonNames', 'isAMdata', 
        'Protein_ID_Formatted', 'isReviewed', 'isReferenceProteome', 'geneSynonyms'
    ]

    # 유지할 컬럼 정의
    cols_to_keep = [col for col in df.columns if col not in cols_to_drop]
    
    print("유지할 컬럼:", cols_to_keep)
    print("삭제할 컬럼:", cols_to_drop)

    # protein 컬럼 가공
    processing_protein_and_save(df)
    print('protein 컬럼을 가공하였습니다.')

    df_main = pd.DataFrame(df[cols_to_keep].copy())

    # 잘 저장되었는지 확인
    print(f"main columns shape: {df_main.shape}")

    df_main.to_csv(main_path, index=False)
    print("데이터가 모두 저장되었습니다.")

In [11]:
delete_columns_and_save(train_data_small, main_path="../data/processed_train_small.csv")
delete_columns_and_save(train_data, main_path="../data/processed_train.csv")

유지할 컬럼: ['Disease ID', 'Disease Name', 'Gene ID', 'GO_Terms', 'PDB_IDs', 'protein1', 'protein2', 'combined_score', 'uniprotAccession', 'proteinFullNames', 'sequence']
삭제할 컬럼: ['PubMed_IDs', 'entryId', 'sequenceChecksum', 'sequenceVersionDate', 'uniprotDescription', 'organismScientificName', 'globalMetricValue', 'uniprotStart', 'uniprotEnd', 'uniprotSequence', 'modelCreatedDate', 'latestVersion', 'allVersions', '_version_', 'proteinShortNames', 'uniprotAccession_unchar', 'entry_name', 'protein_name', 'organism', 'protein_existence', 'sequence_version', 'UniProt_ID', 'uniprotId', 'gene_x', 'gene_y', 'organismScientificNameT', 'tax_id', 'taxId', 'organismCommonNames', 'isAMdata', 'Protein_ID_Formatted', 'isReviewed', 'isReferenceProteome', 'geneSynonyms']
protein 컬럼을 가공하였습니다.
main columns shape: (3402, 11)
데이터가 모두 저장되었습니다.
유지할 컬럼: ['Disease ID', 'Disease Name', 'Gene ID', 'GO_Terms', 'PDB_IDs', 'protein1', 'protein2', 'combined_score', 'uniprotAccession', 'proteinFullNames', 'sequence']
삭

In [12]:
# 데이터 확인
print(train_data_small)

     Disease ID                    Disease Name  Gene ID UniProt_ID  \
0      C0026603                 Motion Sickness    63976     Q9HAZ2   
1      C0035126              Reperfusion Injury     3312     P11142   
2      C0018798        Congenital Heart Defects     4851     P46531   
3      C0004114                     Astrocytoma     4851     P46531   
4      C0024299                        Lymphoma     4851     P46531   
...         ...                             ...      ...        ...   
3397   C0036572                        Seizures    57468     Q9H2X9   
3398   C0024299                        Lymphoma     4851     P46531   
3399   C0948089         Acute Coronary Syndrome     3312     P11142   
3400   C0027627             Neoplasm Metastasis    27436     Q9HC35   
3401   C1960469  Left ventricular noncompaction    63976     Q9HAZ2   

                                               GO_Terms  \
0     GO:0016235; GO:0005829; GO:0005654; GO:0005634...   
1     GO:0072562; GO:0061202;

**이제 다음 feature들만 가지고 학습 진행:**
`index,Disease ID,Disease Name,Gene ID,GO_Terms,PDB_IDs,protein1,protein2,combined_score,uniprotAccession,proteinFullNames,sequence`